# Building a RAG application from scratch
Here is a high-level overview of the system we want to build:

![System1](../images/system1.png)

## Setting up the model

Let's define the LLM model that we'll use as part of the workflow.

In [ ]:
from langchain_ollama.llms import OllamaLLM

try:
    model = OllamaLLM(model="llama3.2")
except ModuleNotFoundError as e:
    print("Please install ollama first.")
    print(e.msg)
    exit()

model.invoke("What is LLM, answer in one sentence.")

The result from the model is an AIMessage instance containing the answer.

We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/how_to/#output-parsers).

Here is what chaining the model with an `output parser` looks like:

![Chain 1](../images/chain1.png)

for above example, you will use simple StrOutputParser to extract the message from object.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
str_output_chain = model | parser
str_output_chain.invoke("What is LLM, answer in one sentence.")

## Introducing prompt templates
We want to provide the model with some context and the question. [Prompt templates]("https://python.langchain.com/docs/how_to/#prompt-templates") are a simple way to define and reuse prompts.

![Chain 2](../images/chain2.png)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=template)
print(
    prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")
)

## Chain prompt with model and output parser

* Input: Context + Question.
* Template Injection: Combines inputs into a structured prompt.
* LLM Processing: The LLM generates an answer based on the injected prompt.
* Output: The answer, which can either be the final result or used in further processing.

![Chain 2](../images/chain2.png)

In [ ]:
chain = prompt | model | parser

chain.invoke(
    input={"context": "Mary's sister is Susana", "question": "Who is Mary's sister?"}
)

## Combining chains

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:
![Chain 3](../images/chain3.png)

In [ ]:
from operator import itemgetter

translation_template = """
    Translate {answer} to {language}.
    Return only translation, without any additional explanation."""
translation_prompt = PromptTemplate(
    input_variables=["answer", "language"], template=translation_template
)

translation_chain = (
    {"answer": chain, "language": itemgetter("language")}
    | translation_prompt
    | model
    | parser
)

translation_chain.invoke(
    {
        "context": "Mary's sister is Susana. She doesn't have any more siblings.",
        "question": "How many sisters does Mary have?",
        "language": "Polish",
    }
)